<a href="https://colab.research.google.com/github/achanhon/coursdeeplearningcolab/blob/master/projet_adversarial_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Projet "apprentissage par ordinateur et exemples adversaires"

Le projet porte sur un problème d'apprentissage "simple" (classification binaire), mais, à travers le prisme des exemples adversaires.

Précisément, le projet s'effectue sur les données http://www.nlpr.ia.ac.cn/pal/trafficdata/recognition.html

- Ce jeu de données permet de faire de la classification multiclasses, mais, il sera rendu binaire pour plus de simplicité: les 16 premières classes seront regroupés (limitation de vitesse et interdiction de tourner) dans la classe 1, tous les autres paneaux sont regroupés dans la classe 2.

- De plus, afin de faciliter le traitement des images, toutes les images seront ramener à des images 96x96.

**Mais** l'objectif n'est pas seulement d'apprendre un classifier (binaire) sur ces images 96x96. L'objectif est 

- de générer des images adversaires pour tromper un tel classifier

**ou**

- d'apprendre un classifier à la fois "robuste" et performant


##prise en main des données

Télécharger les données. Le code ci dessous (**CHEMINS VERS LES DONNEES À MODIFIER**) permet de charger les données i.e. transformer cet ensemble d'image en une grosse matrice (4 matrices préciséments Xtrain,Xtest,Ytrain,Ytest).

In [0]:
import csv
import numpy as np
import PIL
import PIL.Image

print("LOADING DATA")
print("rescaling all images at 96x96")
print("merging classes in 2 super classes (speed limit and turn interdiction vs all)")
Xtrain = np.zeros((4170,3,96,96),dtype=int)
Ytrain = np.zeros(4170,dtype=int)
Xtest = np.zeros((1994,3,96,96),dtype=int)
Ytest = np.zeros(1994,dtype=int)


with open("build/TsignRecgTrain4170Annotation.txt") as f:
    reader = csv.reader(f,delimiter=";")
    
    for i,row in enumerate(reader):
        image3D = np.asarray(PIL.Image.open("build/tsrd-train/"+row[0]).convert("RGB").copy().resize((96,96)),dtype=float)
        Xtrain[i] = np.transpose(image3D,(2,0,1))
        if int(row[7])<16:
            Ytrain[i] = 0
        else:
            Ytrain[i] = 1
        
with open("build/TsignRecgTest1994Annotation.txt") as f:
    reader = csv.reader(f,delimiter=";")
    
    for i,row in enumerate(reader):
        image3D = np.asarray(PIL.Image.open("build/TSRD-Test/"+row[0]).convert("RGB").copy().resize((96,96)),dtype=float)
        Xtest[i] = np.transpose(image3D,(2,0,1))
        if int(row[7])<16:
            Ytest[i] = 0
        else:
            Ytest[i] = 1
        
print("nb of samples with class 0/1 in train=",4170-np.sum(Ytrain),"/",np.sum(Ytrain))
print("nb of samples with class 0/1 in test=",4170-np.sum(Ytest),"/",np.sum(Ytest))

##référence sur les adversariaux

Indépendamment, voilà quelques liens vers des articles de référence sur les adversariaux

* Une introduction au problème des adversariaux : www.tensorflow.org/tutorials/generative/adversarial_fgsm
* L'article correspondant arxiv.org/abs/1412.6572

Du coté de la méthode de référence de "protection" contre les adversariaux
* apprendre sur des données adversaires limites leur effet arxiv.org/abs/1710.10571
* apprendre sur la pire données adversaires possibles permet d'avoir une robustesse certaines arxiv.org/abs/1711.00851

**Important** : générer des données adversaires est très simple quand on a accès au gradient vis à vis de l'image elle même (typiquement si le modèle est du deep learning). Mais c'est possible aussi en approximant le gradient d'un modèle non deep : arxiv.org/pdf/1602.02697.pdf !

##Maintenant à vous de jouer !